This is the notebook where the models are trained.

In [ ]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip -q
!pip install /kaggle/input/keras-applications/ -q
!pip install /kaggle/input/efficientnet-git/ -q

In [ ]:
import json
import math, re, os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import albumentations as A

from functools import partial
from kaggle_datasets import KaggleDatasets

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import (
    Concatenate, Dense, BatchNormalization, 
    GlobalAveragePooling2D, Flatten, Input, 
    Activation, Conv2D, Add, Dropout, GlobalMaxPool2D)
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3

from efficientnet.keras import *

BASE_DIR = "/kaggle/input/cassava-leaf-disease-classification/"
TRAIN_DIR = "/kaggle/input/cassava-leaf-disease-classification/train_images/"
TEST_DIR = "/kaggle/input/cassava-leaf-disease-classification/test_images/"
TRAIN_TF_DIR = "/kaggle/input/cassava-leaf-disease-classification/train_tfrecords/"

sub = pd.read_csv(f'{BASE_DIR}sample_submission.csv')
train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

# Utilities

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum


def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
#     Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
#     image = transform1(image=image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.adjust_contrast(image, 1.5)
    image = tf.image.random_flip_up_down(image)
    
    return image, label

def get_training_dataset(file_names, augmenatate=False):
    dataset = load_dataset(file_names, labeled=True)  
    if augmenatate:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)  
        dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

# Global Setup

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    GCS_PATH = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
    GCS_PATH_AUG = KaggleDatasets().get_gcs_path('cassavaaug')
    
    # Getting the File Names
    TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec')
    TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')
    AUG_FILENAMES = tf.io.gfile.glob(GCS_PATH_AUG + '/*.tfrec')
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas:', strategy.num_replicas_in_sync)


AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
CLASSES = ['0', '1', '2', '3', '4']
EPOCHS = 25

# EDA

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
train["class_name"] = train["label"].map(map_classes)

plt.figure(figsize=(8, 4))
sns.countplot(y="class_name", data=train);

# Modeling

In [ ]:
def efn():
    inputs = Input(shape=(*IMAGE_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')

    # Freeze the pretrained weights
    model.trainable = False
    
    for layer in model.layers[-15:]:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True

#     Rebuild top
    x = GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = BatchNormalization()(x)
    
    x = Dropout(0.5)(x)
    preds = []
    for i in range(12):
        x = Dense(2 ** (i//3+6), kernel_initializer=TruncatedNormal(0, 1e-5, 10211))(x)
        x = BatchNormalization()(x)
        x = Dropout(0.1)(x)
        pred = Dense(5, activation="sigmoid", kernel_initializer=TruncatedNormal(0, 5/(i+1), 231))(x)
        preds.append(pred)
        
    outputs = Concatenate()(preds)
    
    outputs = Dense(5, activation="softmax", 
                    name="prediction", 
                    kernel_initializer=TruncatedNormal(0, 1e-2, 121))(outputs)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNetB0-v4")
    
    optimizer = Adam(learning_rate=9e-4)
    
    
    model_metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="spa")]
    model_loss = tf.keras.losses.SparseCategoricalCrossentropy(name='scc')
    model.compile(
        optimizer=optimizer, loss=model_loss, metrics=model_metrics
    )
    
    return model

In [ ]:
with strategy.scope(): 
    model = efn()
        
#     model = tf.keras.models.load_model('/kaggle/input/nncassava/efnet05-01.h5')
#     tf.keras.backend.set_value(model.optimizer.learning_rate, 1e-4)  
#     model.optimizer.learning_rate = 1e-4

# model.summary()

In [ ]:
# model.summary()

# Traininig

In [ ]:
hist = model.fit(
            get_training_dataset(TRAINING_FILENAMES),
            epochs=800,
            verbose=1,
            callbacks = [
                EarlyStopping(monitor='spa', 
                              patience=100, 
                              restore_best_weights=True, 
                              min_delta=1e-3, 
                              verbose=True,
                              mode='max'
                             ),
                ReduceLROnPlateau(monitor='spa',
                                  factor=0.95, 
                                  patience=5, 
                                  verbose=0,
                                  mode='max',
                                  min_delta=1e-3, 
                                  cooldown=0,
                                  min_lr=1e-8)
            ]
        )

# Sanity Check

In [ ]:
pd.DataFrame(hist.history).loc[:, ['spa']].plot(title="Accuracy")

In [ ]:
model.save('/kaggle/working/efnet01-05.h5')

# Submission

In [ ]:
testing_dataset = get_test_dataset()

def to_float32(image, label):
    return tf.cast(image, tf.float32), label
test_ds = get_test_dataset(ordered=True) 
test_ds = test_ds.map(to_float32)


print('Computing predictions...')    
test_images_ds = testing_dataset
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

NUM_TEST_IMAGES = len(TEST_FILENAMES)
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
!head submission.csv